<a href="https://colab.research.google.com/github/BaoNguyen151654/Database_ML_Bikeshop_Project/blob/master/ML_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

In [2]:
df=pd.read_csv('/content/Data_for_ML.csv')
df

,customer_id,total_spent,latest_purchased_brand,latest_brand_tier,most_purchased_brand,most_purchased_brand_tier,latest_brand_name,most_purchased_brand_name
0,94,34807.9392,9,high,9,high,Trek,Trek
1,94,34807.9392,9,high,9,high,Trek,Trek
2,94,34807.9392,9,high,9,high,Trek,Trek
3,94,34807.9392,9,high,9,high,Trek,Trek
4,94,34807.9392,9,high,9,high,Trek,Trek
...,...,...,...,...,...,...,...,...
4228,177,170.9910,9,high,9,high,Trek,Trek
4229,752,170.9910,9,high,9,high,Trek,Trek
4230,1111,170.9910,9,high,9,high,Trek,Trek
4231,718,167.9920,2,mid,2,mid,Haro,Haro


In [3]:
df.drop(columns=['latest_brand_name', 'most_purchased_brand_name','customer_id'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4233 entries, 0 to 4232
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   total_spent                4233 non-null   float64
 1   latest_purchased_brand     4233 non-null   int64  
 2   latest_brand_tier          4233 non-null   object 
 3   most_purchased_brand       4233 non-null   int64  
 4   most_purchased_brand_tier  4233 non-null   object 
dtypes: float64(1), int64(2), object(2)
memory usage: 165.5+ KB


In [4]:
tier_mapping = {'high': 3, 'mid': 2, 'low': 1}
df['latest_brand_tier'] = df['latest_brand_tier'].map(tier_mapping)
df['most_purchased_brand_tier'] = df['most_purchased_brand_tier'].map(tier_mapping)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4233 entries, 0 to 4232
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   total_spent                4233 non-null   float64
 1   latest_purchased_brand     4233 non-null   int64  
 2   latest_brand_tier          4233 non-null   int64  
 3   most_purchased_brand       4233 non-null   int64  
 4   most_purchased_brand_tier  4233 non-null   int64  
dtypes: float64(1), int64(4)
memory usage: 165.5 KB


In [6]:
df.corr()

,total_spent,latest_purchased_brand,latest_brand_tier,most_purchased_brand,most_purchased_brand_tier
total_spent,1.000000,0.220416,0.229604,0.301518,0.306126
latest_purchased_brand,0.220416,1.000000,0.715499,0.458983,0.355934
latest_brand_tier,0.229604,0.715499,1.000000,0.325423,0.409084
most_purchased_brand,0.301518,0.458983,0.325423,1.000000,0.789185
most_purchased_brand_tier,0.306126,0.355934,0.409084,0.789185,1.000000


In [7]:
value_counts = df['latest_brand_tier'].value_counts()
value_counts

,count
latest_brand_tier,
2,2283
3,1709
1,241


In [8]:
X = df.drop(columns=['latest_brand_tier'])
y = df['latest_brand_tier']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

smote = SMOTE(sampling_strategy={1: 1200}, random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

weights = {1: 3, 2: 1, 3: 1}
model = LogisticRegression(
    penalty='elasticnet',
    l1_ratio=0.4,
    class_weight=weights,
    solver='saga',
    max_iter=1000
)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred)

In [9]:
print(report)

              precision    recall  f1-score   support

           1       0.35      0.95      0.51        65
           2       0.97      0.75      0.85       709
           3       0.86      0.93      0.89       496

    accuracy                           0.83      1270
   macro avg       0.72      0.88      0.75      1270
weighted avg       0.89      0.83      0.85      1270

